In [1]:
#!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 7.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=5c8d9c0a6b3f1202806ccb48f5c522e45d3e59748037f6ec4e5cafaef339c748
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [30]:
#Без этого костыля у меня не работал
import os
import sys

os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'

In [31]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ml-iris').getOrCreate()
df = spark.read.csv('1.csv', header = True, inferSchema = True)
df.printSchema()

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- variety: string (nullable = true)



In [32]:
import pandas as pd
pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
sepal_length,5.1,4.9,4.7,4.6,5.0
sepal_width,3.5,3.0,3.2,3.1,3.6
petal_length,1.4,1.4,1.3,1.5,1.4
petal_width,0.2,0.2,0.2,0.2,0.2
variety,Setosa,Setosa,Setosa,Setosa,Setosa


In [33]:
#dfs = pd.read_csv('/content/1.csv')

In [34]:
#dfs.head()

In [35]:
#f = dfs.columns

In [36]:
#f[:-1]

In [37]:
#dfs.rename(columns = {'sepal.length':'sepal_length', 'sepal.width':'sepal_width', 
#                     'petal.length':'petal_length', 'petal.width':'petal_width'}, inplace = True )

In [38]:
#dfs.to_csv('1.csv', index=False)

In [39]:
# numeric_features = [t[0] for t in df.dtypes if t[1] == 'double']
# df.select(numeric_features).describe().toPandas().transpose()

In [40]:
from pyspark.ml.feature import StringIndexer, VectorAssembler

In [41]:
numericCols = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
assembler = VectorAssembler(inputCols=numericCols, outputCol="features")
df = assembler.transform(df)
df.show()

+------------+-----------+------------+-----------+-------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|variety|         features|
+------------+-----------+------------+-----------+-------+-----------------+
|         5.1|        3.5|         1.4|        0.2| Setosa|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2| Setosa|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2| Setosa|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2| Setosa|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2| Setosa|[5.0,3.6,1.4,0.2]|
|         5.4|        3.9|         1.7|        0.4| Setosa|[5.4,3.9,1.7,0.4]|
|         4.6|        3.4|         1.4|        0.3| Setosa|[4.6,3.4,1.4,0.3]|
|         5.0|        3.4|         1.5|        0.2| Setosa|[5.0,3.4,1.5,0.2]|
|         4.4|        2.9|         1.4|        0.2| Setosa|[4.4,2.9,1.4,0.2]|
|         4.9|        3.1|         1.5|        0.1| Setosa|[4.9,

In [42]:
label_stringIdx = StringIndexer(inputCol = 'variety', outputCol = 'labelIndex')
df = label_stringIdx.fit(df).transform(df)
df.show(10)

+------------+-----------+------------+-----------+-------+-----------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|variety|         features|labelIndex|
+------------+-----------+------------+-----------+-------+-----------------+----------+
|         5.1|        3.5|         1.4|        0.2| Setosa|[5.1,3.5,1.4,0.2]|       0.0|
|         4.9|        3.0|         1.4|        0.2| Setosa|[4.9,3.0,1.4,0.2]|       0.0|
|         4.7|        3.2|         1.3|        0.2| Setosa|[4.7,3.2,1.3,0.2]|       0.0|
|         4.6|        3.1|         1.5|        0.2| Setosa|[4.6,3.1,1.5,0.2]|       0.0|
|         5.0|        3.6|         1.4|        0.2| Setosa|[5.0,3.6,1.4,0.2]|       0.0|
|         5.4|        3.9|         1.7|        0.4| Setosa|[5.4,3.9,1.7,0.4]|       0.0|
|         4.6|        3.4|         1.4|        0.3| Setosa|[4.6,3.4,1.4,0.3]|       0.0|
|         5.0|        3.4|         1.5|        0.2| Setosa|[5.0,3.4,1.5,0.2]|       0.0|
|         4.4|       

In [45]:
pd.DataFrame(df.take(110), columns=df.columns).transpose()

,0,1,2,3,4,5,6,7,8,9,...,100,101,102,103,104,105,106,107,108,109
sepal_length,5.1,4.9,4.7,4.6,5.0,5.4,4.6,5.0,4.4,4.9,...,6.3,5.8,7.1,6.3,6.5,7.6,4.9,7.3,6.7,7.2
sepal_width,3.5,3.0,3.2,3.1,3.6,3.9,3.4,3.4,2.9,3.1,...,3.3,2.7,3.0,2.9,3.0,3.0,2.5,2.9,2.5,3.6
petal_length,1.4,1.4,1.3,1.5,1.4,1.7,1.4,1.5,1.4,1.5,...,6.0,5.1,5.9,5.6,5.8,6.6,4.5,6.3,5.8,6.1
petal_width,0.2,0.2,0.2,0.2,0.2,0.4,0.3,0.2,0.2,0.1,...,2.5,1.9,2.1,1.8,2.2,2.1,1.7,1.8,1.8,2.5
variety,Setosa,Setosa,Setosa,Setosa,Setosa,Setosa,Setosa,Setosa,Setosa,Setosa,...,Virginica,Virginica,Virginica,Virginica,Virginica,Virginica,Virginica,Virginica,Virginica,Virginica
features,"[5.1, 3.5, 1.4, 0.2]","[4.9, 3.0, 1.4, 0.2]","[4.7, 3.2, 1.3, 0.2]","[4.6, 3.1, 1.5, 0.2]","[5.0, 3.6, 1.4, 0.2]","[5.4, 3.9, 1.7, 0.4]","[4.6, 3.4, 1.4, 0.3]","[5.0, 3.4, 1.5, 0.2]","[4.4, 2.9, 1.4, 0.2]","[4.9, 3.1, 1.5, 0.1]",...,"[6.3, 3.3, 6.0, 2.5]","[5.8, 2.7, 5.1, 1.9]","[7.1, 3.0, 5.9, 2.1]","[6.3, 2.9, 5.6, 1.8]","[6.5, 3.0, 5.8, 2.2]","[7.6, 3.0, 6.6, 2.1]","[4.9, 2.5, 4.5, 1.7]","[7.3, 2.9, 6.3, 1.8]","[6.7, 2.5, 5.8, 1.8]","[7.2, 3.6, 6.1, 2.5]"
labelIndex,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [46]:
train, test = df.randomSplit([0.7, 0.3], seed = 2018)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 103
Test Dataset Count: 47


In [47]:
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression

In [41]:
train

DataFrame[sepal_length: double, sepal_width: double, petal_length: double, petal_width: double, variety: string, features: vector, labelIndex: double]

In [50]:
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'labelIndex')
rfModel = rf.fit(train)
predictions = rfModel.transform(test)
predictions.select('sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'labelIndex', 'rawPrediction', 'prediction', 'probability').show(25)

+------------+-----------+------------+-----------+----------+--------------------+----------+--------------------+
|sepal_length|sepal_width|petal_length|petal_width|labelIndex|       rawPrediction|prediction|         probability|
+------------+-----------+------------+-----------+----------+--------------------+----------+--------------------+
|         4.3|        3.0|         1.1|        0.1|       0.0|      [20.0,0.0,0.0]|       0.0|       [1.0,0.0,0.0]|
|         4.6|        3.6|         1.0|        0.2|       0.0|      [20.0,0.0,0.0]|       0.0|       [1.0,0.0,0.0]|
|         4.8|        3.0|         1.4|        0.1|       0.0|      [20.0,0.0,0.0]|       0.0|       [1.0,0.0,0.0]|
|         4.8|        3.0|         1.4|        0.3|       0.0|      [20.0,0.0,0.0]|       0.0|       [1.0,0.0,0.0]|
|         4.9|        2.4|         3.3|        1.0|       1.0|      [0.0,18.0,2.0]|       1.0|       [0.0,0.9,0.1]|
|         4.9|        3.1|         1.5|        0.1|       0.0|      [20.

In [51]:
predictions.select("labelIndex", "prediction").show(10)

+----------+----------+
|labelIndex|prediction|
+----------+----------+
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       0.0|       0.0|
+----------+----------+
only showing top 10 rows



In [53]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [54]:
evaluator = MulticlassClassificationEvaluator(labelCol="labelIndex", predictionCol="prediction")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %s" % (accuracy))
print("Test Error = %s" % (1.0 - accuracy))

Accuracy = 0.9361702127659575
Test Error = 0.06382978723404253


In [55]:
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType
import pyspark.sql.functions as F

preds_and_labels = predictions.select(['prediction','labelIndex']).withColumn('labelIndex', F.col('labelIndex').cast(FloatType())).orderBy('prediction')
preds_and_labels = preds_and_labels.select(['prediction','labelIndex'])
metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))
print(metrics.confusionMatrix().toArray())

c:\Users\andru\anaconda3\lib\site-packages\pyspark\sql\context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


[[12.  0.  0.]
 [ 0. 16.  1.]
 [ 0.  2. 16.]]


In [56]:
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel

In [39]:
spark.conf.set("spark.sql.ansi.enabled", "false")

In [59]:
#model_path = "./rfc_model"
#rfModel.write().overwrite().save(model_path)
#model2 = RandomForestClassificationModel.load(model_path)
